# Python-1, лекция 14

**Лектор:** Петров Тимур

## HTTP и иже с ним

Теперь приступим к новым вещам, а именно - к вебом и взаимодействием с ним!

![](https://www.thebozho.com/wp-content/uploads/2017/04/Free-Open-Internet.jpg)

Давайте начнем просто с того, что же такое HTTP? В переводе это HyperText Transfer Protocol, или же протокол для передачи т.н. HyperText (гиепртекст - это текст с наличием ссылок для перехода на другие странички, изи)

Что в таком случае https, который мы сейчас везде видим? Это тот же HTTP, но еще и S (secure) - данные передаются не просто открыто, а через протокол шифрования [TLS](https://habr.com/ru/post/258285/) (не будет вдаваться в подробности, как оно работает, речь не про это)

По сути своей, это модель "клиент-сервер". Клиент дает запрос, сервер отвечает, еще запрос, еще ответ etc хоть до бесконечности. Выглядит максимально просто и понятно

К примеру, когда вы заходите на сайт, вы отправляете запрос по типу "покажи сайт". Сервер такой: "Держи". Потом тыкаете на ссылку, опять запрос etc



Что происходит при открытии сайта?

Например, вы заходите в [питон](https://python.org/). Что необходимо сделать для этого браузеру?

Браузер отправляет запрос на сервер, который расположен по адресу python.org. Удивительно, но это можно сделать даже без браузера (кто бы мог подумать)

In [1]:
# !brew install telnet
# !telnet python.org 80 # утилита, которая осуществляет запрос как клиентская часть

/bin/bash: line 1: brew: command not found
Trying 151.101.64.223...
Connected to python.org.
Escape character is '^]'.
^C
Connection closed by foreign host.


Подсоединились, теперь нам надо отправлять запросы, чтобы получать какие-то ответы. Как выглядят запросы в HTTP?

На самом деле тут все достаточно просто. Любой запрос в HTTP выглядит следующим образом:

```
Метод URI HTTP/1.1\n
Host: hostname\n
\n

```

* Метод - собственно операция, которую хотим осуществить

* URI - путь до документа/страницы, которая нам нужна

* HTTP/1.1 - указываем версию HTTP

* hostname - название хоста (так как удалённый сервер не обладает никакой информацией о том, какой именно адрес использовался для соединения, то его надо указывать)

На самом деле можно указывать также дополнительные параметры (например, какой у нас User-Agent (браузер) etc)

Что мы всегда видим в качестве ответа? В первую очередь строку следующего вида:

```
HTTP/1.1 Код Пояснение
```

* HTTP/1.1 - версия HTTP

* Код - [код выполнения запроса](https://developer.mozilla.org/ru/docs/Web/HTTP/Status)

Вот наши любимые:

1. 200 - все ок

2. 4** - вы лохи (сюда относится и 403, 404 - запрещено/не найдено)

3. 5** - мы лохи

* Пояснение - название кода ответа (типа OK, Not Found etc)


Какие есть основные методы для HTTP?

* GET - получи инфу

* POST - запость инфу

* PUT - обнови инфу

* DELETE - удали инфу

Если вы решили, что можно взять и тогда удалить Python, то вот нет. Единственное, что обязан обрабатывать веб-сервер - это GET. Остального может не быть (и тогда сервер вас просто не поймет). А еще могут быть всякие другие методы

Понятное дело, что для этого должна быть какая-нибудь удобная библиотека (а не тупо telnet, в который еще как-то странно надо передавать данные). Для этого есть requests!

## Requests

[Requests](https://requests.readthedocs.io/en/latest/api/) - это отличная библиотека для работы с подключениями к сайту. Давайте разбираться на примерах:

In [ ]:
import requests

requests.get('https://api.github.com') #аналогия get, получаем ответ

Какие атрибуты есть у get?

In [3]:
r = requests.get('https://api.github.com')
print(r.status_code, r.reason) # статус ответа (200 - все ОК)
print('-' * 30)
for i in r.headers.items(): # информация
    print(i[0], ':', i[1])
print('-' * 30)
print(r.encoding)
print('-' * 30)
print(r.text) # что вывели (тело) (в чистом виде можно с помощью .content, text сразу декодирует)
print('-' * 30)
print(r.url) # где находимся
print('-' * 30)
print(r.json()) # и можно сразу в виде json

200 OK
------------------------------
Date : Fri, 06 Sep 2024 12:53:16 GMT
Content-Type : application/json; charset=utf-8
Cache-Control : public, max-age=60, s-maxage=60
Vary : Accept,Accept-Encoding, Accept, X-Requested-With
ETag : W/"4f825cc84e1c733059d46e76e6df9db557ae5254f9625dfe8e1b09499c449438"
X-GitHub-Media-Type : github.v3; format=json
x-github-api-version-selected : 2022-11-28
Access-Control-Expose-Headers : ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset
Access-Control-Allow-Origin : *
Strict-Transport-Security : max-age=31536000; includeSubdomains; preload
X-Frame-Options : deny
X-Content-Type-Options : nosniff
X-XSS-Protection : 0
Referrer-Policy : origin-when-cross-origin, strict-origin-when-cross-origin
Content-Security-Policy : default-sr

Иногда мы хотим делать get не просто по сайту, а еще передавать какие-нибудь параметры. Давайте попробуем:

In [4]:
response = requests.get(
    'https://api.github.com/search/repositories',
    params={'q': 'Deep_Python'}, #делаем поиск с помощью параметра q https://docs.github.com/en/rest/search
)

# Анализ некоторых атрибутов местонахождения запросов
json_response = response.json()
repository = json_response['items'][0]
print(f'Repository name: {repository["name"]}')
print(f'Repository description: {repository["description"]}')

Repository name: deep-learning-with-python-notebooks
Repository description: Jupyter notebooks for the code samples of the book "Deep Learning with Python"


In [5]:
for i in range(len(json_response['items'])):
    if json_response['items'][i]['name'] == 'Deep_Python':
        print(i)

16


In [6]:
json_response['items'][12]

{'id': 145148726,
 'node_id': 'MDEwOlJlcG9zaXRvcnkxNDUxNDg3MjY=',
 'name': 'MachineLearningNotebooks',
 'full_name': 'Azure/MachineLearningNotebooks',
 'private': False,
 'owner': {'login': 'Azure',
  'id': 6844498,
  'node_id': 'MDEyOk9yZ2FuaXphdGlvbjY4NDQ0OTg=',
  'avatar_url': 'https://avatars.githubusercontent.com/u/6844498?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/Azure',
  'html_url': 'https://github.com/Azure',
  'followers_url': 'https://api.github.com/users/Azure/followers',
  'following_url': 'https://api.github.com/users/Azure/following{/other_user}',
  'gists_url': 'https://api.github.com/users/Azure/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/Azure/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/Azure/subscriptions',
  'organizations_url': 'https://api.github.com/users/Azure/orgs',
  'repos_url': 'https://api.github.com/users/Azure/repos',
  'events_url': 'https://api.github.com/users/Azure/events{/

Какие есть функции?

Есть самый основные для HTTP-протокола:

In [7]:
requests.post('https://httpbin.org/post', data={'key':'value'}) #запостить инфу
requests.put('https://httpbin.org/put', data={'key':'value'}) #полная замена инфы (придется указать все, что даже не меняется)
requests.delete('https://httpbin.org/delete') #удалить инфу
requests.head('https://httpbin.org/get') #получить информацию без тела
requests.patch('https://httpbin.org/patch', data={'key':'value'}) #замена инфы, но без указания того, что не надо заменять

<Response [200]>

Иногда для того, чтобы получить доступ, надо сделать авторизацию (аутентификацию), для этого внутри requests есть

In [8]:
from requests.auth import HTTPBasicAuth #самый простой способ аутентификации, почти везде принимается
from getpass import getpass #библиотека из 2 функций: запрос пароля и запрос пользователя

requests.get(
     'https://api.github.com/user',
     auth=HTTPBasicAuth('user', getpass())
)

KeyboardInterrupt: Interrupted by user

Давайте себе представим ситуацию: сервер подвис! Что же делать, наш код будет сидеть и ждать, а такого мы не хотим. Внутри requests можно задать время ожидания, который мы используем

In [ ]:
from requests.exceptions import Timeout

try:
    response = requests.get('https://api.github.com', timeout=1) #устанавливаем максимальный timeout, если не случилось, то он бросает Timeout
except Timeout:
    print('The request timed out')
else:
    print('The request did not time out')

И последнее, что мы сегодня разберем, так это Session. Что это такое?

Сессия - это возможность сделать несколько обращений, при этом сохранив общие параметры, которые вам нужны (ну, например, если у вас вдруг везде одинаковый логин-пароль, можно его сохранить и использовать для нескольких вызовов)

In [ ]:
with requests.Session() as session:
    session.auth = ('username', getpass())
    session.headers.update({'one': 'true'})
    print(session.headers)
    response = session.get('https://api.github.com')

print(response.headers)
print(response.json())

## BeautifulSoup4

И начнем с самого простого: парсинг обычных html страниц. Для этого есть прекрасная бибилотека [BeautifulSoup4](https://beautiful-soup-4.readthedocs.io/en/latest/)

И в качестве примера возьмем вики нашего факультета)

### Основы

In [ ]:
import requests

r = requests.get('http://wiki.cs.hse.ru/Заглавная_страница')
page = r.content.decode("utf-8")
page ##выглядит как каша какая-то

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page, 'html.parser') # указываем парсер
print(soup.prettify()) # выглядит уже более структурно

Здесь есть аргумент parser, что это такое? Внутри BeautifulSoup есть несколько различных парсеров, которые можно использовать (в зависимости сложности документа)

* html.parser - самый простой и достаточно быстрй парсер, включенный уже внутри BS (чаще всего ничего более не требуется для HTML-документов), а еще он менее строгий к документу (то есть игнорирует всякую фигню)

* lxml - парсер из другой бибилотеки (тоже в разделе про XML)

* xml - парсер для XML документов (про них позже)

* html5lib - парсит как реальный веб-браузер это делает (но его еще отдельно надо установить, работает медленно), очень строгий к формату

Базово, что мы хотим от парсера? Ходить по тэгам и вытягивать оттуда информацию. В каком-то смысле для этого не требуется знать HTML-язык разметки, но в качестве общего ознакомления почитать про [тэги](https://developer.mozilla.org/ru/docs/Web/HTML) следует

Давайте вытянем название странички:

In [ ]:
print(soup.title) #навигация идет по названиям тэгов, например, здесь взяли тэг title
print(soup.title.name) # получили название тэга
print(soup.title.string) # получили текст внутри тэга - бинго!

А теперь давайте разбираться вообще в структуре HTML-странички в целом:

- Все, что находится внутри < и > - это тэг. Тэг сам по себе задает различные действия (например, < a > - это ссылка, < h1 > - заголовок первого уровня, самый большой)

Все, что внутри тэга - это его контент, то есть то, к чему относится данный тэг. А также у тэга могут быть свои аттрибуты

На примере:

```
        <p>
         <a href="/%D0%9B%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D0%B0%D1%8F_%D0%B0%D0%BB%D0%B3%D0%B5%D0%B1%D1%80%D0%B0_%D0%B8_%D0%B3%D0%B5%D0%BE%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D1%8F_%D0%BD%D0%B0_%D0%9F%D0%9C%D0%98_2022/2023_(%D0%BF%D0%B8%D0%BB%D0%BE%D1%82%D0%BD%D1%8B%D0%B9_%D0%BF%D0%BE%D1%82%D0%BE%D0%BA)" title="Линейная алгебра и геометрия на ПМИ 2022/2023 (пилотный поток)">
          Линейная алгебра и геометрия (пилотный поток)
         </a>
        </p>
```

Что видим? Тэг p - абзац, тэг a - ссылка. То есть получаем абзац с текстом "Линейная алгебра и геометрия (пилотный поток)", на котором лежит ссылка с адресом href

Внутри тэга a есть два аттрибута: href и title (ссылка и название)

![](https://younghtml5studynote.files.wordpress.com/2011/12/1.jpg)


Базово внутри любого HTML-документа есть head, body и footer. (Вообще такие тэги появились только в HTML-5, раньше делали просто div)

- **Head** - заглавие (где указывается название сайта, а так же отрисовки, стиль, скрипты etc)

- **Body** - основное тело документа, где лежит вся информация

- **Footer** - это все, что находится внизу. Многие не любят делать footer отдельно, а тупо вгоняют его в body, выделяя отдельно место (с помощью div)

Также скрипты могут встречаться в любом случае, если надо что-то подятнуть

Это вот такая база для понимания, как страница устроена

Нас, наверное, не очень интересует header и footer, так как хотим доставать основную информацию. Давайте по коду определим, где находится то, что нам нужно.

Оно находится внутри ```<div id="bodyContent"> ```. Давайте туда и перейдем

In [11]:
root = soup.find(id="bodyContent") #с помощью функции find можно искать необходимые тэги, id.
# Если какой-то более сложный атрибут, то можно передать через словарь attrs

### Ищем данные

Давайте теперь вытянем все названия второго уровня с сайта (то есть это на сайте будет "Курсы за 2022/23 год", "Курсы в рамках проекта Data Culture" etc)

Все они находятся в тэгах < h2 >. Значит надо найти все такие тэги и вывести текст!

Для этого есть функция ```find_all```

In [12]:
root.find_all('h2')[0].contents #Вывели самый первый такой, но при этом он еще в какой-то мишуре в виде span

[<span class="mw-headline" id=".D0.9A.D1.83.D1.80.D1.81.D1.8B_.D0.B7.D0.B0_2024.2F25_.D1.83.D1.87.D0.B5.D0.B1.D0.BD.D1.8B.D0.B9_.D0.B3.D0.BE.D0.B4"><span id="bachelors">Курсы за 2024/25 учебный год</span></span>]

In [13]:
root.find_all('h2')[0].span.string # Хопа, прошли по span и вывели сам текст!

'Курсы за 2024/25 учебный год'

In [14]:
root.find_all('h2') # Вытянули все значения, осталось проитерироваться и достать все

[<h2><span class="mw-headline" id=".D0.9A.D1.83.D1.80.D1.81.D1.8B_.D0.B7.D0.B0_2024.2F25_.D1.83.D1.87.D0.B5.D0.B1.D0.BD.D1.8B.D0.B9_.D0.B3.D0.BE.D0.B4"><span id="bachelors">Курсы за 2024/25 учебный год</span></span></h2>,
 <h2><span class="mw-headline" id=".D0.9A.D1.83.D1.80.D1.81.D1.8B_.D0.B2_.D1.80.D0.B0.D0.BC.D0.BA.D0.B0.D1.85_.D0.BF.D1.80.D0.BE.D0.B5.D0.BA.D1.82.D0.B0_Data_Culture"><span id="DataCulture">Курсы в рамках проекта <a class="external text" href="https://www.hse.ru/dataculture/" rel="nofollow">Data Culture</a></span></span></h2>,
 <h2><span class="mw-headline" id=".D0.9A.D1.83.D1.80.D1.81.D1.8B_.D0.BC.D0.B0.D0.B3.D0.B8.D1.81.D1.82.D1.80.D0.B0.D1.82.D1.83.D1.80.D1.8B_.D0.A4.D0.9A.D0.9D"><span id="masters">Курсы магистратуры ФКН</span></span></h2>,
 <h2><span class="mw-headline" id=".D0.9A.D1.83.D1.80.D1.81.D1.8B_.D0.B4.D1.80.D1.83.D0.B3.D0.B8.D1.85_.D1.84.D0.B0.D0.BA.D1.83.D0.BB.D1.8C.D1.82.D0.B5.D1.82.D0.BE.D0.B2"><span id="other">Курсы других факультетов</span></span></

In [15]:
for k in root.find_all('h2'):
    print(k.span.contents) # Почему берем именно contents? Потому что если внутри есть еще один тэг, то это дополнительно дробление (a)

[<span id="bachelors">Курсы за 2024/25 учебный год</span>]
[<span id="DataCulture">Курсы в рамках проекта <a class="external text" href="https://www.hse.ru/dataculture/" rel="nofollow">Data Culture</a></span>]
[<span id="masters">Курсы магистратуры ФКН</span>]
[<span id="other">Курсы других факультетов</span>]
['Курсы за 2023/24 учебный год']
['Курсы в рамках проекта ', <a class="external text" href="https://www.hse.ru/dataculture/" rel="nofollow">Data Culture</a>]
['Курсы магистратуры ФКН']
['Курсы других факультетов']
['Факультативы']


В чем проблема? Так в том, что тут еще есть ссылки, из которых надо тоже все это вытягивать:

In [16]:
for k in root.find_all('h2'):
    res = ""
    if k.a is not None:
        res += k.span.contents[0]
        res += k.span.a.string
    else:
        res += k.span.string
    print(res)

Курсы за 2024/25 учебный год


TypeError: can only concatenate str (not "Tag") to str

### Поиск текста и ссылок

Ура, получилось! Чуть-чуть глиномесно, но представьте делать это совсем руками...

Хорошо, допустим, что теперь мы хотим просто получить весь текст с сайта, как бы сделать?

In [17]:
print(root.stripped_strings) # получаем генератор, который очищает от тэгов и внутри них ищет текст
print(root.strings)

<generator object PageElement.stripped_strings at 0x7fd8ec6572a0>
<generator object Tag._all_strings at 0x7fd8ec480d60>


В чем разница?

In [20]:
print(next(root.stripped_strings)) # убирает всякие переносы строк, табуляицю etc
print(next(root.strings))

Давайте теперь все напечатаем:

In [ ]:
import requests
from bs4 import BeautifulSoup

website = input()
line = input()

r = requests.get('http://wiki.cs.hse.ru/Заглавная_страница')
page = r.content.decode("utf-8")


soup = BeautifulSoup(page, 'html.parser') # указываем парсер
for text in root.stripped_strings:
    print(text)

In [21]:
for text in root.stripped_strings:
    print(text)

Материал из Wiki - Факультет компьютерных наук
(перенаправлено с «
Заглавная страница
»)
Перейти к:
навигация
,
поиск
Учебные курсы факультета компьютерных наук
Навигация
Курсы бакалавриата ФКН
ПМИ
·
ПИ
·
ПАД
·
КНАД
·
ДРИП
·
майноры и факультативы
Курсы в рамках проекта Data Culture
·
Курсы магистратуры ФКН
·
Курсы других факультетов
·
Архив
Курсы за 2024/25 учебный год
1 курс
2 курс
3 курс
4 курс
майноры и факультативы
ПМИ
М+
Линейная алгебра и геометрия (пилотный поток)
Математический анализ-1 (пилотный поток)
Дискретная математика (пилотный поток)
Алгебра (пилотный поток)
Теория чисел (пилотный поток)
М
Линейная алгебра и геометрия (основной поток)
Математический анализ-1 (ПМИ основной поток)
Математический анализ-1 (ЭАД)
Дискретная математика (ПМИ основной поток)
Дискретная математика (ЭАД+СмолГУ)
Алгебра (основной поток)
Теория чисел (основной поток)
П+
Алгоритмы и структуры данных (пилотный поток)
П
Язык программирования Python 2024/25 (основной поток)
Алгоритмы и структуры данны

А теперь хотим найти ссылки - достаточн частая задача.

Ссылки (кликабельные) всегда находятся в тэге a. Поэтому давайте икать в них атрибут href (который задает ссылку)

Внутри любого тэга можно достать атрибут с помощью get():

In [22]:
for a in root.find_all('a'):
    print(a.get('href'))

/index.php?title=%D0%97%D0%B0%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D0%B0%D1%8F_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D0%B0&redirect=no
#mw-navigation
#p-search
#bachelors
#AMI
#SE
#DSBA
#compds
#DRIP
#electives
#DataCulture
#masters
#other
#archive
/%D0%9B%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D0%B0%D1%8F_%D0%B0%D0%BB%D0%B3%D0%B5%D0%B1%D1%80%D0%B0_%D0%B8_%D0%B3%D0%B5%D0%BE%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D1%8F_%D0%BD%D0%B0_%D0%9F%D0%9C%D0%98_2024/2025_(%D0%BF%D0%B8%D0%BB%D0%BE%D1%82%D0%BD%D1%8B%D0%B9_%D0%BF%D0%BE%D1%82%D0%BE%D0%BA)
/index.php?title=%D0%9C%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_1_2024/25_(%D0%BF%D0%B8%D0%BB%D0%BE%D1%82%D0%BD%D1%8B%D0%B9_%D0%BF%D0%BE%D1%82%D0%BE%D0%BA)&action=edit&redlink=1
/index.php?title=%D0%94%D0%B8%D1%81%D0%BA%D1%80%D0%B5%D1%82%D0%BD%D0%B0%D1%8F_%D0%BC%D0%B0%D1%82%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B0_%D0%BD%D0%B0_%D0%9F%D0%9C%D0%98_2024/25_(%D0%BF%D0%B8%D0%BB%D0%BE%D1%82%D0%BD%D1%8

В чем проблема? В том, что мы находим None (есть тэг, но нет ссылки) и что есть навигация по внутренним страничкам (Вики же)

Давайте оставим именно ссылки:

In [23]:
for a in root.find_all('a'):
    if a.get('href') is not None and 'http' in a.get('href'):
        print(a.get('href'))

https://hse24.fmin.xyz
https://osukhoroslov-hse.notion.site/2da6d9d4dc5944e59414daea40376d96
https://osukhoroslov-hse.notion.site/2-013e88a10c204059bff194f838de4ef5?pvs=4
https://www.hse.ru/dataculture/
https://hse24.fmin.xyz
https://osukhoroslov-hse.notion.site/2da6d9d4dc5944e59414daea40376d96
https://osukhoroslov-hse.notion.site/2-013e88a10c204059bff194f838de4ef5?pvs=4
https://www.hse.ru/dataculture/
http://wiki.cs.hse.ru/index.php?title=Wiki_ФКН&oldid=85870


### Дополнительно

Что еще умеем? Хотим по родителям, детям и соседям (вверх, вниз, вправо-влево)

In [ ]:
k = root.find("a")
print(root.find("a"))
print(k.next_element) #следующий элемент
print(k.previous_element) #прошлый элемент
print(k.parent) # внутри какого уровня находимя
print(k.child) #что внутри по уровню

Отдельно есть функция select, с помощью которой можно задавать более конкретно, что мы ищем:

In [ ]:
root.select("h2 > span") #найти все тэги h2, внутри которых лежит span

In [ ]:
soup.select("#right-navigation") #поиск по id

Можно усилить BeautifulSoup: там можно и изменять HTML-код (добавлять-удалять-менять тэги, аттрибуты etc)

Но главное, что BS умеет работать с паттернами регулярок и ходят бок о бок!

## Попугай дня

![](https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Rose-ringed_parakeet_%28Psittacula_krameri_manillensis%29.jpg/1024px-Rose-ringed_parakeet_%28Psittacula_krameri_manillensis%29.jpg)

А это ожереловый попугай (или индийский кольчатый попугай). Этот вид явно выделяет наличие такого колечка на шее. Самый распространенный попугай в мире

Они очень хорошо и быстро летают, но ходят максимально неуклюже)

В отличии от многих попугаев, для которых человеческая деятельность является вредом и уменьшает их естественный ареал обитания, то как раз для этих птиц человеческая деятельность - это плюс, потому что они находят корм внутри человеческих обществ (не боятся людей, лучше всего приспосабливаются, едят пшеницу). Поговаривают, что в качестве домашних попугаев их держали еще в древней Греции и Риме, а в  средневековой Индии ожереловые попугаи часто были питомцами знатных особ и монархов, причём владение этим попугаем считалось элементом роскоши и престижа